In [11]:
import pandas as pd
import requests
from mapbox import Geocoder

# Sample DataFrame creation for demonstration purposes
# Replace this with your actual DataFrame loading code
data = {'location': ['fenway', 'times square', 'eiffel tower']}
df = pd.DataFrame(data)

# Assuming your DataFrame is loaded correctly from the CSV file
# df = pd.read_csv('your_file_name.csv')

# Replace 'YOUR_MAPBOX_ACCESS_TOKEN' with your actual Mapbox access token
mapbox_access_token = 'pk.eyJ1IjoiYWJkdWxyYWZheTAzIiwiYSI6ImNsdHB5YzRmczAwZTAyaXFiMnd6MXBjbXQifQ.fy4hqXUdVEsmPuC5XLyUSA'

geocoder = Geocoder(access_token=mapbox_access_token)

# Modified function to return the TRACT information
def query_census_api(longitude, latitude):
    url = f"https://geocoding.geo.census.gov/geocoder/geographies/coordinates?x={longitude}&y={latitude}&benchmark=Public_AR_Current&vintage=Census2020_Current&format=json"
    response = requests.get(url)
    if response.status_code == 200:
        results = response.json()
        census_tracts = results['result']['geographies'].get('Census Tracts', [])
        if census_tracts:
            # Returning the TRACT of the first census tract found
            return census_tracts[0].get('TRACT', 'No TRACT found')
    return "No TRACT found"  # Return this if API call failed or no tracts found

# Initialize an empty list to store TRACT information
tract_list = []

# Iterate over each row in the DataFrame
for location in df['location']:
    response = geocoder.forward(location)
    if response.status_code == 200:
        # Extract the coordinates of the first matching feature
        coordinates = response.json()['features'][0]['geometry']['coordinates']
        
        # Query the U.S. Census Geocoding Services API with these coordinates
        TRACT = query_census_api(coordinates[0], coordinates[1])
        tract_list.append(TRACT)
    else:
        print(f"Failed to get coordinates for {location}")
        tract_list.append("Failed to get TRACT")

# Adding the TRACT information as a new column to the DataFrame
df['TRACT'] = tract_list

df

,location,TRACT
0,fenway,010405
1,times square,No TRACT found
2,eiffel tower,006700
